# GARCH Models
**FinRisk Analytics - Project 2**

Fit GARCH, EGARCH, GJR-GARCH models.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

from src.data.fetch_data import load_data
from src.data.preprocess import prepare_returns, align_returns
from src.models.garch_models import GARCHVolatility, EGARCHVolatility, GJRGARCHVolatility, compare_garch_models
from src.utils.plotting import plot_volatility, plot_model_comparison, ensure_plot_dir

with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('✅ Ready for GARCH modeling!')

In [ ]:
# Load returns data
case_studies = config['assets']['case_studies']
price_data = {}

for ticker in case_studies:
    df = load_data(ticker, data_dir='../data/raw')
    if df is not None:
        price_data[ticker] = df

returns_dict = prepare_returns(price_data)
returns_df = align_returns(returns_dict)
returns_df.head()

In [ ]:
## Model Comparison
# Compare GARCH family models on each asset

ensure_plot_dir('../results/plots')

for ticker in returns_df.columns:
    print(f"\n{'='*60}")
    print(f"Fitting GARCH models for {ticker}")
    print('='*60)
    
    returns = returns_df[ticker]
    
    # Compare models
    comparison = compare_garch_models(returns)
    print(f"\nModel Comparison for {ticker}:")
    print(comparison)
    
    # Fit each model and get forecasts
    garch = GARCHVolatility()
    garch.fit(returns)
    
    egarch = EGARCHVolatility()
    egarch.fit(returns)
    
    gjr = GJRGARCHVolatility()
    gjr.fit(returns)
    
    print(f"\n✅ Models fitted for {ticker}")

In [ ]:
## Rolling Volatility Forecasts
# Generate rolling forecasts using GARCH

ticker = 'SPY'  # Focus on SPY as example
returns = returns_df[ticker]

print(f"Generating rolling forecasts for {ticker}...")

# GARCH forecasts
garch = GARCHVolatility()
garch_forecasts = garch.rolling_forecast(returns, window=252, refit_freq=20)

# EGARCH forecasts
egarch = EGARCHVolatility()
egarch_forecasts = egarch.rolling_forecast(returns, window=252, refit_freq=20)

# Calculate realized volatility
realized_vol = returns.rolling(window=20).std() * np.sqrt(252)

# Plot comparison
predictions = {
    'GARCH': garch_forecasts,
    'EGARCH': egarch_forecasts
}

fig, ax = plot_model_comparison(
    realized_vol.loc[garch_forecasts.index],
    predictions,
    title=f'{ticker} - Volatility Forecast Comparison',
    save_path=f'../results/plots/{ticker}_garch_comparison.png'
)
plt.show()

print("\n✅ GARCH Analysis Complete!")